<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.1
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [3]:
import requests #this is used for HTTP request
import json #this is used for parsing json data
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [4]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code and text of the response:

In [5]:
#ANSWER
response.status_code

200

In [6]:
# it prints the status of the response (200 if correct)

In [7]:
print(response.text)

{"timestamp": 1721290340, "message": "success", "iss_position": {"latitude": "-16.7464", "longitude": "87.4884"}}


In [ ]:
#ANSWER


{"iss_position": {"latitude": "-30.3403", "longitude": "-135.9143"}, "message": "success", "timestamp": 1656486612}


We can use another API to request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21).

In [8]:
response = requests.get("https://api.g7vrd.co.uk/v1/satellite-passes/25544/-33.87/151.21.json?minelevation=0&hours=24")

Print the response header:

In [9]:
#ANSWER - Response header
print(response.headers)


{'Date': 'Thu, 18 Jul 2024 08:18:36 GMT', 'Server': 'Apache', 'Vary': 'Origin,Access-Control-Request-Method,Access-Control-Request-Headers', 'Access-Control-Allow-Origin': '*', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '0', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'X-Frame-Options': 'DENY', 'Content-Type': 'application/json', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked'}


Print the content of the response (the data that the server returned):

In [11]:
#ANSWER - Content of the response
print(response.content)

b'{"api_status":"ALPHA","request_timestamp":"2024-07-18T08:18:36.477752565Z","norad_id":25544,"satellite_name":"ISS","tle_last_retrieved":"2024-07-17T10:06:18.033694159Z","lat":-33.87,"lon":151.21,"hours":24,"min_elevation":0,"query_ms":20,"passes":[{"start":"2024-07-18T16:53:41.458Z","tca":"2024-07-18T16:56:11.458Z","end":"2024-07-18T16:58:51.458Z","aos_azimuth":30,"los_azimuth":87,"max_elevation":3.0},{"start":"2024-07-18T18:26:51.458Z","tca":"2024-07-18T18:32:21.458Z","end":"2024-07-18T18:37:46.458Z","aos_azimuth":323,"los_azimuth":128,"max_elevation":54.0},{"start":"2024-07-18T20:04:21.458Z","tca":"2024-07-18T20:09:21.458Z","end":"2024-07-18T20:14:16.458Z","aos_azimuth":273,"los_azimuth":146,"max_elevation":16.0},{"start":"2024-07-18T21:43:56.458Z","tca":"2024-07-18T21:47:26.458Z","end":"2024-07-18T21:50:36.458Z","aos_azimuth":228,"los_azimuth":153,"max_elevation":4.0},{"start":"2024-07-18T23:22:31.458Z","tca":"2024-07-18T23:26:01.458Z","end":"2024-07-18T23:29:06.458Z","aos_azimuth

In [13]:
print(type(response.content)) # result is the type of content

<class 'bytes'>


Note that this is a Python byte string:

In [ ]:
print(type(response.content))

Print just the "content-type" value from the header:

In [15]:
#ANSWER
print(response.headers['content-type'])


application/json


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [16]:
overheads = response.json()
print(overheads)

{'api_status': 'ALPHA', 'request_timestamp': '2024-07-18T08:18:36.477752565Z', 'norad_id': 25544, 'satellite_name': 'ISS', 'tle_last_retrieved': '2024-07-17T10:06:18.033694159Z', 'lat': -33.87, 'lon': 151.21, 'hours': 24, 'min_elevation': 0, 'query_ms': 20, 'passes': [{'start': '2024-07-18T16:53:41.458Z', 'tca': '2024-07-18T16:56:11.458Z', 'end': '2024-07-18T16:58:51.458Z', 'aos_azimuth': 30, 'los_azimuth': 87, 'max_elevation': 3.0}, {'start': '2024-07-18T18:26:51.458Z', 'tca': '2024-07-18T18:32:21.458Z', 'end': '2024-07-18T18:37:46.458Z', 'aos_azimuth': 323, 'los_azimuth': 128, 'max_elevation': 54.0}, {'start': '2024-07-18T20:04:21.458Z', 'tca': '2024-07-18T20:09:21.458Z', 'end': '2024-07-18T20:14:16.458Z', 'aos_azimuth': 273, 'los_azimuth': 146, 'max_elevation': 16.0}, {'start': '2024-07-18T21:43:56.458Z', 'tca': '2024-07-18T21:47:26.458Z', 'end': '2024-07-18T21:50:36.458Z', 'aos_azimuth': 228, 'los_azimuth': 153, 'max_elevation': 4.0}, {'start': '2024-07-18T23:22:31.458Z', 'tca': '2

What kind of object did this give us?

In [17]:
#ANSWER:
print(type(overheads))

<class 'dict'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `passes` value to a separate list:

In [18]:
#ANSWER:
passes = overheads['passes']

In [19]:
passes

[{'start': '2024-07-18T16:53:41.458Z',
  'tca': '2024-07-18T16:56:11.458Z',
  'end': '2024-07-18T16:58:51.458Z',
  'aos_azimuth': 30,
  'los_azimuth': 87,
  'max_elevation': 3.0},
 {'start': '2024-07-18T18:26:51.458Z',
  'tca': '2024-07-18T18:32:21.458Z',
  'end': '2024-07-18T18:37:46.458Z',
  'aos_azimuth': 323,
  'los_azimuth': 128,
  'max_elevation': 54.0},
 {'start': '2024-07-18T20:04:21.458Z',
  'tca': '2024-07-18T20:09:21.458Z',
  'end': '2024-07-18T20:14:16.458Z',
  'aos_azimuth': 273,
  'los_azimuth': 146,
  'max_elevation': 16.0},
 {'start': '2024-07-18T21:43:56.458Z',
  'tca': '2024-07-18T21:47:26.458Z',
  'end': '2024-07-18T21:50:36.458Z',
  'aos_azimuth': 228,
  'los_azimuth': 153,
  'max_elevation': 4.0},
 {'start': '2024-07-18T23:22:31.458Z',
  'tca': '2024-07-18T23:26:01.458Z',
  'end': '2024-07-18T23:29:06.458Z',
  'aos_azimuth': 205,
  'los_azimuth': 132,
  'max_elevation': 4.0},
 {'start': '2024-07-19T00:58:56.458Z',
  'tca': '2024-07-19T01:03:56.458Z',
  'end': '2024

Now extract the `start` strings into an array called `srisetimes`:

In [20]:
#ANSWER:
srisetimes = [xpass['start'] for xpass in passes]

In [21]:
srisetimes

['2024-07-18T16:53:41.458Z',
 '2024-07-18T18:26:51.458Z',
 '2024-07-18T20:04:21.458Z',
 '2024-07-18T21:43:56.458Z',
 '2024-07-18T23:22:31.458Z',
 '2024-07-19T00:58:56.458Z',
 '2024-07-19T02:35:21.458Z',
 '2024-07-19T04:14:06.458Z',
 '2024-07-19T17:38:46.458Z']

These are strings. We convert these to an array of Python `datetime` values called `risetimes`:

In [22]:
risetimes = [datetime.strptime(xpass['start'], "%Y-%m-%dT%H:%M:%S.%fZ") for xpass in passes]
risetimes

[datetime.datetime(2024, 7, 18, 16, 53, 41, 458000),
 datetime.datetime(2024, 7, 18, 18, 26, 51, 458000),
 datetime.datetime(2024, 7, 18, 20, 4, 21, 458000),
 datetime.datetime(2024, 7, 18, 21, 43, 56, 458000),
 datetime.datetime(2024, 7, 18, 23, 22, 31, 458000),
 datetime.datetime(2024, 7, 19, 0, 58, 56, 458000),
 datetime.datetime(2024, 7, 19, 2, 35, 21, 458000),
 datetime.datetime(2024, 7, 19, 4, 14, 6, 458000),
 datetime.datetime(2024, 7, 19, 17, 38, 46, 458000)]

Finally, use `risetime.strftime` to print these in a format that people understand:

```
e.g.
18/10/22 07:05
18/10/22 08:41
18/10/22 10:20
18/10/22 12:00
18/10/22 01:37
18/10/22 03:13
```



In [23]:
#ANSWER:
for risetime in risetimes:
    print(risetime.strftime('%d/%m/%y %I:%M'))

18/07/24 04:53
18/07/24 06:26
18/07/24 08:04
18/07/24 09:43
18/07/24 11:22
19/07/24 12:58
19/07/24 02:35
19/07/24 04:14
19/07/24 05:38


Finally, here is an endpoint that tells us who is on board:

In [24]:
response = requests.get("http://api.open-notify.org/astros.json")

In [25]:
response

<Response [200]>

In [26]:
print(response.text)

{"people": [{"craft": "ISS", "name": "Oleg Kononenko"}, {"craft": "ISS", "name": "Nikolai Chub"}, {"craft": "ISS", "name": "Tracy Caldwell Dyson"}, {"craft": "ISS", "name": "Matthew Dominick"}, {"craft": "ISS", "name": "Michael Barratt"}, {"craft": "ISS", "name": "Jeanette Epps"}, {"craft": "ISS", "name": "Alexander Grebenkin"}, {"craft": "ISS", "name": "Butch Wilmore"}, {"craft": "ISS", "name": "Sunita Williams"}, {"craft": "Tiangong", "name": "Li Guangsu"}, {"craft": "Tiangong", "name": "Li Cong"}, {"craft": "Tiangong", "name": "Ye Guangfu"}], "number": 12, "message": "success"}


Referring to the methods used above, extract the number of astronauts and their names:

In [29]:
#ANSWER:
astros = response.json()
print(astros)
print(astros['number'])
for astronaut in astros['people']:
    print(astronaut['name'])

{'people': [{'craft': 'ISS', 'name': 'Oleg Kononenko'}, {'craft': 'ISS', 'name': 'Nikolai Chub'}, {'craft': 'ISS', 'name': 'Tracy Caldwell Dyson'}, {'craft': 'ISS', 'name': 'Matthew Dominick'}, {'craft': 'ISS', 'name': 'Michael Barratt'}, {'craft': 'ISS', 'name': 'Jeanette Epps'}, {'craft': 'ISS', 'name': 'Alexander Grebenkin'}, {'craft': 'ISS', 'name': 'Butch Wilmore'}, {'craft': 'ISS', 'name': 'Sunita Williams'}, {'craft': 'Tiangong', 'name': 'Li Guangsu'}, {'craft': 'Tiangong', 'name': 'Li Cong'}, {'craft': 'Tiangong', 'name': 'Ye Guangfu'}], 'number': 12, 'message': 'success'}
12
Oleg Kononenko
Nikolai Chub
Tracy Caldwell Dyson
Matthew Dominick
Michael Barratt
Jeanette Epps
Alexander Grebenkin
Butch Wilmore
Sunita Williams
Li Guangsu
Li Cong
Ye Guangfu


## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [ ]:
#ANSWER:
def handleResponse(response, verbose = False):
    '''
    Returns Boolean Value, Status Code,
    '''
  # if Status Code is 200 return false, and status code
  # Otherwise Return True and Status Code

2. Test your response handler on some correct and incorrect API calls.

In [ ]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')

response = requests.get("http://api.open-notify.org/iss-now.json")
handleResponse(response, True)[0]

>

>

>



---



---



> > > > > > > > > © 2024 Institute of Data


---



---



